In [1]:
import pyspark
import pandas as pd
import os
import csv
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler,StringIndexer
spark = SparkSession.builder.getOrCreate()

In [2]:
df = spark.createDataFrame(
    pd.read_csv('C:/Users/nares/Downloads/iris.data', header=None, names=['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class'])
    )

In [3]:
df.show()

+------------+-----------+------------+-----------+-----------+
|sepal-length|sepal-width|petal-length|petal-width|      class|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|
|         5.4|        3.9|         1.7|        0.4|Iris-setosa|
|         4.6|        3.4|         1.4|        0.3|Iris-setosa|
|         5.0|        3.4|         1.5|        0.2|Iris-setosa|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|
|         4.9|        3.1|         1.5|        0.1|Iris-setosa|
|         5.4|        3.7|         1.5|        0.2|Iris-setosa|
|         4.8|        3.4|         1.6|        0.2|Iris-setosa|
|         4.8|        3.0|         1.4| 

In [4]:
feature_cols = df.columns[:-1]
assembler = pyspark.ml.feature.VectorAssembler(inputCols=feature_cols, outputCol='features')
df = assembler.transform(df)

In [5]:
df = df.select(['features', 'class'])
label_indexer = pyspark.ml.feature.StringIndexer(inputCol='class', outputCol='label').fit(df)
df = label_indexer.transform(df)
df = df.select(['features', 'label'])
df.show(10)

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[5.1,3.5,1.4,0.2]|  0.0|
|[4.9,3.0,1.4,0.2]|  0.0|
|[4.7,3.2,1.3,0.2]|  0.0|
|[4.6,3.1,1.5,0.2]|  0.0|
|[5.0,3.6,1.4,0.2]|  0.0|
|[5.4,3.9,1.7,0.4]|  0.0|
|[4.6,3.4,1.4,0.3]|  0.0|
|[5.0,3.4,1.5,0.2]|  0.0|
|[4.4,2.9,1.4,0.2]|  0.0|
|[4.9,3.1,1.5,0.1]|  0.0|
+-----------------+-----+
only showing top 10 rows



In [6]:
train, test = df.randomSplit([0.70, 0.30])
lr = pyspark.ml.classification.LogisticRegression(regParam=0.01)
model = lr.fit(train)

In [7]:
prediction = model.transform(test)
print("Prediction")
prediction.show(10)

Prediction
+-----------------+-----+--------------------+--------------------+----------+
|         features|label|       rawPrediction|         probability|prediction|
+-----------------+-----+--------------------+--------------------+----------+
|[4.3,3.0,1.1,0.1]|  0.0|[6.57678564610645...|[0.98617765341388...|       0.0|
|[5.4,3.7,1.5,0.2]|  0.0|[6.03590886582174...|[0.98115252127876...|       0.0|
|[5.7,4.4,1.5,0.4]|  0.0|[6.72385262477322...|[0.99530300276438...|       0.0|
|[5.1,3.8,1.5,0.3]|  0.0|[6.43473115591857...|[0.99096730705439...|       0.0|
|[5.2,3.4,1.4,0.2]|  0.0|[5.74929824475867...|[0.96997519900804...|       0.0|
|[5.2,3.5,1.5,0.2]|  0.0|[5.86857180954486...|[0.97574279268458...|       0.0|
|[5.4,3.4,1.5,0.4]|  0.0|[4.96309128500362...|[0.94191908105509...|       0.0|
|[5.4,3.4,1.7,0.2]|  0.0|[5.19605902728392...|[0.94434748492885...|       0.0|
|[5.0,3.5,1.3,0.3]|  0.0|[6.11251549820776...|[0.98417829068817...|       0.0|
|[5.1,3.8,1.6,0.2]|  0.0|[6.5512150270587

In [8]:
pred_data = spark.createDataFrame(
[(5.1, 3.5, 1.4, 0.2),
(6.2, 3.4, 5.4, 2.3)],
["sepal_length", "sepal_width", "petal_length", "petal_width"])

In [9]:
feature_vals = pred_data.columns[:]
assembler = pyspark.ml.feature.VectorAssembler(inputCols=feature_vals, outputCol='features')
pred_data = assembler.transform(pred_data)

In [10]:
pred_data = pred_data.select(['features'])
prediction = model.transform(pred_data)
prediction = prediction.select('prediction').toPandas()

In [11]:
os.chdir(r'C:/Users/nares/Downloads')
outlst = prediction.values.tolist()
f = open('out_3_2.csv', 'w', encoding='utf-8',newline='')
head = 'class'
f.write(head+'\n')
for i in outlst:
    f.write(str(i[0])+'\n')
f.close()